# Osculating circle
A commuter which moves with constant velocity $v$ on a curve $f(x)$:

$ \dot{x} = v \cos{\theta}$ 
$ \dot{y} = v \sin{\theta}$

$ f'(x) = \frac{dy}{dx} = \tan{\theta}$


$ \ddot{x} = -v \sin{\theta} \dot{\theta}$ 
$ \ddot{y} = v \cos{\theta} \dot{\theta}$


$ \ddot{y} = f''\dot{x}^2 + f' \ddot{x} \Rightarrow  \dot{\theta} = f'' v \cos^3{\theta} $

$ a_r = \frac{v^2 f''}{(1+f'^2)^{3/2}} $

$ \rho = \frac{v}{ \dot{\theta}}$

In [1]:
import numpy as np
from scipy.integrate import quad
from scipy.misc import derivative


from bokeh.io import output_notebook, push_notebook
from bokeh.models import ColumnDataSource
from bokeh.models.markers import Cross
from bokeh.plotting import figure, show
import time

output_notebook()

/home/arash/miniconda3/envs/dril/lib/python3.6/site-packages/scipy/__init__.py:115: UserWarning: Numpy 1.13.3 or above is required for this version of scipy (detected version 1.11.3)
  UserWarning)


Loading BokehJS ...

In [2]:
# the curve function
def f(x):
    return np.sin(x)+np.cos(x*2)+x

# it's first derivative: computed automatically
def df(x):
    return derivative(f, x, dx=1e-6)

# it's second derivative: computed automatically
def ddf(x):
    return derivative(f, x, n=2, dx=1e-6)

In [3]:
# volocity of the commuter on the curve
V = 1 
# the length of the vectors
L = 1

def theta(x):
    """
    computes the slope angle
    """
    try:
        return np.arctan(df(x))
    except:
        return 0.
    
def theta_dot(x):
    """
    computes the rate of change of slop's angle
    """
    th = theta(x)
    return ddf(x)*V*np.cos(th)**3
   
def rho(x):
    """
    computes the curvature radius
    """
    if theta_dot(x)!=0:
        return V/theta_dot(x)
    else:
        return 10

def osculating_circle_position(x):
    """
    computes the center of the center of the osculating circle
    """
    y = f(x)
    th = theta(x)
    r = rho(x)
    return ( x-r*np.sin(th), y+r*np.cos(th) )
   
def compute_position(x, t):
    """
    computes the position of the commuter on the cureve 
    at time t by integration.
    """
    func = lambda x: V/np.sqrt(1+df(x)**2)
    return quad(func,0, t)[0]

In [4]:
# plot the curve
X = np.arange(0,20,.2)
Y = f(X)

# time array
dt =  np.pi/100
times = np.arange(0,5*np.pi, dt)

# setting up the sources
t = 0
x = X[0]
y = Y[0]
th = theta(x)
center = osculating_circle_position(x)

# this source contains the postions of commuter and circle's center
source = ColumnDataSource(data=dict(x=[x], y=[y], radius = [rho(x)], \
                                    center_x=[center[0]], center_y=[center[1]], ))

# this source contains the vectors' starting and end points
vector_src = ColumnDataSource(data={'x_tangent': [x,x+L*np.cos(th)], 'y_tangent': [y,y+L*np.sin(th)],\
                                    'x_radial' : [x,center[0]], 'y_radial': [y,center[1]]})


# the figure limits: figure must be isometric
dY = Y.max()-Y.min()
dX = X.max()-X.min()
if dX>dY:
    x_start = X.min() - .2*dX
    x_end = X.max() + .2*dX
    y_start = Y.mean()- 0.7*dX
    y_end = Y.mean()+ 0.7*dX
else:
    y_start = Y.min() - .2*dY
    y_end = Y.max() + 0.2*dY
    x_start = X.mean()- 0.7*dY
    x_end = X.mean()+ 0.7*dY

    
# setting up the figure
plot = figure( title ='Time: '+str(t), x_range=(x_start,x_end), y_range=(y_start,y_end))

plot.line(X, Y, line_width=3, line_alpha=0.5,)
point = plot.circle('x','y',  source=source, size=4)
circle = plot.circle('center_x','center_y', radius = 'radius',  source=source, fill_color='green', fill_alpha=0.5 , color ='green')
radial_arrow= plot.line('x_radial','y_radial', line_width=3, line_alpha=0.5, source=vector_src, color='black')
tangent_arrow= plot.line('x_tangent','y_tangent', line_width=3, line_alpha=0.5, source=vector_src, color='black')
glyph = Cross(x="center_x", y="center_y", line_color="red", fill_color=None, line_width=2, size=5)
plot.add_glyph(source, glyph)

handle=show(plot, notebook_handle=True);

# starting of the animation
while t<=times[-1]:
    
    x = point.data_source.data['x']
    y = point.data_source.data['y']
    
    new_x = compute_position(x, t)
    new_y = f(new_x)
    new_th = theta(new_x)
    new_r = rho(new_x)
    circle_data = osculating_circle_position(new_x)
    t += dt 
    
    point.data_source.data['x'] = [new_x]
    point.data_source.data['y'] = [new_y]
    circle.data_source.data['center_x'] = [circle_data[0]]
    circle.data_source.data['center_y'] = [circle_data[1]]
    circle.data_source.data['radius'] = [new_r]
        
    radial_arrow.data_source.data['x_radial'] = [new_x, new_x +(circle_data[0]-new_x)/(new_r**2)]
    radial_arrow.data_source.data['y_radial'] = [new_y, new_y +(circle_data[1]-new_y)/(new_r**2)]
    radial_arrow.data_source.data['x_tangent'] = [new_x, new_x+L*np.cos(new_th)]
    radial_arrow.data_source.data['y_tangent'] = [new_y, new_y+L*np.sin(new_th)]
    
    plot.title.text = 'Time: {0:.2f}'.format(t)
    if t==times[-1]:
        t=0
        x=0
    
    # push updates to the plot continuously using the handle (intererrupt the notebook kernel to stop)
    push_notebook(handle=handle)
    time.sleep(0.1)

KeyboardInterrupt: 